# AMR catalogue demo
This notebook should show how to utilise different AMR catalogues within `piezo` and `gnomon` to produce predictions.


## Piezo
`piezo` allows for loading of an AMR catalogue, and producing predictions from mutations

In [3]:
#This is a basic demo of how piezo works
import piezo

#Load the NEJM catalogue
catalogue = piezo.ResistanceCatalogue("tuberculosis_amr_catalogues/catalogues/NC_000962.3/NC_000962.3_NEJM2018_v1.1_GARC1_RUS.csv")

#Get the catalogue's prediction for rpoB@S450L
catalogue.predict("rpoB@S450L")


{'RIF': 'R'}

## Gumpy
`gumpy` is a genetics library, which allows loading of reference genomes and samples (via VCF) to produce helpful outputs such as genome variants and gene mutations.

In [4]:
#Basic demo of how gumpy works
import gumpy

#Load a reference genome (this takes ~40s)
reference = gumpy.Genome("tuberculosis_amr_catalogues/catalogues/NC_000962.3/NC_000962.3.gbk")

In [5]:
#Load a VCF
#In this case, a VCF resulting from a synthetic sample which has known mutations
vcf = gumpy.VCFFile("XDR_demo.vcf")

#Get the sample genome
sample = reference + vcf

#Get the difference
diff = reference - sample

#Pull out the genome level variants
print(diff.variants)

#Get the gene level mutations (simplified here as we already know the genes with mutations)
#In reality, iterating the genes can be used to detect all mutations
genes = ['rpoB', 'gyrA', 'rplC']
mutations = []
for gene in genes:
    diff = reference.build_gene(gene) - sample.build_gene(gene)
    print(gene, diff.mutations)
    for mutation in diff.mutations:
        mutations.append(gene + "@" + mutation)

['7570c>t' '7585g>c' '761155c>t' '801268t>c']
rpoB ['S450L']
gyrA ['A90V' 'S95T']
rplC ['C154R']


In [6]:
#Utilise the mutations found by gumpy to produce predictions from the catalogue
for mutation in mutations:
    print(mutation, catalogue.predict(mutation))

rpoB@S450L {'RIF': 'R'}
gyrA@A90V S
gyrA@S95T S
rplC@C154R S


In [7]:
#If we decide that another catalogue is better, its easy to swap out
#Let's try with the WHO catalogue
catalogue = piezo.ResistanceCatalogue("tuberculosis_amr_catalogues/catalogues/NC_000962.3/WHO-UCN-GTB-PCI-2021.7.GARC.csv")
for mutation in mutations:
    print(mutation, catalogue.predict(mutation))

rpoB@S450L {'RIF': 'R'}
gyrA@A90V {'LEV': 'R', 'MXF': 'R'}
gyrA@S95T {'LEV': 'S', 'MXF': 'S'}
rplC@C154R {'LZD': 'R'}


## Gnomon
Above shows how simple this makes the process programatically, but `gnomon` can be utilised to produce these automatically through a single command line call, as well as providing an API for programatic interface.

In [8]:
#Produce an antibiogram in a single command line call
!!gnomon --genome_object tuberculosis_amr_catalogues/catalogues/NC_000962.3/NC_000962.3.gbk --catalogue tuberculosis_amr_catalogues/catalogues/NC_000962.3/WHO-UCN-GTB-PCI-2021.7.GARC.csv --vcf_file XDR_demo.vcf --output_dir ./gnomon-out --json

['zsh:1: command not found: gnomon']

In [9]:
#Checking the output files produced by gnomon
import pandas as pd

#The genome variants
print(pd.read_csv("gnomon-out/XDR_demo.variants.csv"))

   UNIQUEID    VARIANT  IS_SNP  NUCLEOTIDE_INDEX  IS_INDEL  IS_NULL  IS_HET  \
0  XDR_demo    7570c>t    True              7570     False    False   False   
1  XDR_demo    7585g>c    True              7585     False    False   False   
2  XDR_demo  761155c>t    True            761155     False    False   False   
3  XDR_demo  801268t>c    True            801268     False    False   False   

   INDEL_LENGTH  INDEL_NUCLEOTIDES  
0             0                NaN  
1             0                NaN  
2             0                NaN  
3             0                NaN  


In [10]:
#The gene mutations
print(pd.read_csv("gnomon-out/XDR_demo.mutations.csv"))

   UNIQUEID  GENE MUTATION  NUCLEOTIDE_NUMBER  NUCLEOTIDE_INDEX  \
0  XDR_demo  gyrA     A90V                NaN               NaN   
1  XDR_demo  gyrA     S95T                NaN               NaN   
2  XDR_demo  rplC    C154R                NaN               NaN   
3  XDR_demo  rpoB    S450L                NaN               NaN   

   GENE_POSITION  ALT  REF  CODES_PROTEIN  INDEL_LENGTH  ...  IS_CDS  IS_HET  \
0             90  gtg  gcg           True           NaN  ...    True   False   
1             95  acc  agc           True           NaN  ...    True   False   
2            154  cgt  tgt           True           NaN  ...    True   False   
3            450  ttg  tcg           True           NaN  ...    True   False   

   IS_NULL  IS_PROMOTER  IS_SNP  AMINO_ACID_NUMBER  AMINO_ACID_SEQUENCE  \
0    False        False    True                 90                    V   
1    False        False    True                 95                    T   
2    False        False    True       

In [11]:
#The antibiogram
print(pd.read_csv("gnomon-out/XDR_demo.effects.csv"))

   UNIQUEID DRUG  GENE MUTATION          CATALOGUE_NAME PREDICTION
0  XDR_demo  LEV  gyrA     A90V  WHO-UCN-GTB-PCI-2021.7          R
1  XDR_demo  MXF  gyrA     A90V  WHO-UCN-GTB-PCI-2021.7          R
2  XDR_demo  LEV  gyrA     S95T  WHO-UCN-GTB-PCI-2021.7          S
3  XDR_demo  MXF  gyrA     S95T  WHO-UCN-GTB-PCI-2021.7          S
4  XDR_demo  LZD  rplC    C154R  WHO-UCN-GTB-PCI-2021.7          R
5  XDR_demo  RIF  rpoB    S450L  WHO-UCN-GTB-PCI-2021.7          R


In [17]:
#The output JSON
import json
out = json.load(open("gnomon-out/XDR_demo.gnomon-out.json"))

#Get the effects of mutations on levofloxacin, as well as predicted phenotype
print(json.dumps(out['data']['EFFECTS']['LEV'], indent=2))

[
  {
    "GENE": "gyrA",
    "MUTATION": "A90V",
    "PREDICTION": "R"
  },
  {
    "GENE": "gyrA",
    "MUTATION": "S95T",
    "PREDICTION": "S"
  },
  {
    "PHENOTYPE": "R"
  }
]


## More advanced
As `gnomon` writes output files, we can go back to samples previously processed and produce new predictions with a new catalogue without having to actually re-process entirely. This kind of processing is likely not worthwhile for a single sample, but avoiding generating mutations will provide significant speed up for multiple samples.

In [18]:
import gnomon
import os

#We need a dict of reference genes
genes = ['rpoB', 'gyrA', 'rplC']
referenceGenes = {gene: reference.build_gene(gene) for gene in genes}

#The VCF stem (which is used as a sample ID)
vcfStem = os.path.split("XDR_demo.vcf")[-1].split(".")[0]
outputDir = 'gnomon-out'

#Read the mutations produced by gnomon the last time
mutations = pd.read_csv("gnomon-out/XDR_demo.mutations.csv")

#Re-make the effects using a new catalogue (in this case, the NEJM catalogue)
catalogue = piezo.ResistanceCatalogue("tuberculosis_amr_catalogues/catalogues/NC_000962.3/NC_000962.3_NEJM2018_v1.1_GARC1_RUS.csv")

effects, _ = gnomon.populateEffects(None, outputDir, catalogue, mutations, referenceGenes, vcfStem)

print(effects)

#Update the JSON output with this too
variants = pd.read_csv("gnomon-out/XDR_demo.variants.csv")

gnomon.saveJSON(variants, mutations, effects, "gnomon-out", vcfStem, "RUS", "1.0.0")

100%|██████████| 4/4 [00:00<00:00, 1563.87it/s]

   UNIQUEID DRUG  GENE MUTATION CATALOGUE_NAME PREDICTION
0  XDR_demo  RIF  rpoB    S450L       NEJM2018          R
